In [76]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ast import literal_eval
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import os
import sympy as sym
from sympy.parsing import mathematica
import numpy as np

In [77]:
plt.rcParams['text.usetex'] = False

In [78]:

RESULTS_DIR = "paper_results"
benchmarks = [
    "rational_2",
    "sqrt",    
    "rational",
    "rational_sqrt",   
]

benchmarks_id= {
    
    "rational":'a',
    "rational_2":'b',
    "sqrt":'c',
    "rational_sqrt":'d',
}


In [79]:
def process_output(row):
    try:
        if (type(row) == list) and (len(row) > 0):
            row = np.asarray(row)
            result = (pd.to_numeric(row.mean() - row.std()), pd.to_numeric(row.mean() + row.std()))
            print("Returning from: Check if row is a non-empty list")
            return result
    except:
        pass

    try:
        if (type(row) == list) and (len(row) == 0):
            result = (None, None)
            print("Returning from: Check if row is an empty list")
            return result
    except:
        pass

    try:
        row = float(row)
        result = (row, row)
        print("Returning from: Convert row to float")
        return result
    except:
        pass

    try:
        result = pd.to_numeric(literal_eval(str(row).strip()))
        print("Returning from: Use literal_eval on row")
        return result
    except Exception as e:
        pass

    try:
        for w in ['integrate', 'boole', 'throw', 'complex']:
            if w in row.lower():
                result = (None, None)
                print("Returning from: Parse and evaluate row as a symbolic expression (contains forbidden words)")
                return result
        
        row = row.replace("[", '(').replace("]", ")").replace("L", "l").replace("E(p_) =", "").replace("^", "**").strip()
        row = sym.parse_expr(row).evalf()
        result = (row, row)
        print("Returning from: Parse and evaluate row as a symbolic expression")
        return result
    except Exception as e:
        print(e, row)
    
    print("Returning from: Default case (None, None)")
    return (None, None)

In [80]:
def process_time(row):
    if pd.isnull(row):
        return None

    try:
        row = float(row)
        return row
    except:
        pass

    try:
        return literal_eval(str(row).strip())[0]
    except Exception as e:
        pass
        
    except Exception as e:
        print(e, row)
    
        
    return None

In [81]:
def process_benchmark(benchmark):
    df_list = []
    for f_name in os.listdir(os.path.join(RESULTS_DIR, benchmark)):
        tool_name =  f_name.split("_")[-2]
        df_ = pd.read_csv(os.path.join(RESULTS_DIR, benchmark, f_name))
        df_['tool'] = tool_name
        df_['details'] = df_['details'].apply(literal_eval)
        if tool_name == 'volesti': 
            df_['output'] = df_['details'].apply(lambda d: list([x['output'] for x in d]))    
        df_list.append(df_[['index', 'tool','output', 'time', 'details']])
    df = pd.concat(df_list, ignore_index=True)

    df['raw_output'] = df['output']
    df['output'] = df['output'].apply(process_output)
    df['time'] = df['time'].apply(process_time)
    
    df['lower'], df['upper'] = df['output'].apply(lambda o: o[0]),df['output'].apply(lambda o: o[1])
    df['error'] = df['upper']-df['lower']
    df['error_norm'] = MinMaxScaler().fit_transform(df['error'].values.reshape(-1,1))
    df['benchmark'] = benchmark
    df['id'] = df['index'].apply(lambda x: benchmarks_id[benchmark]+str(x))
    df['is_timeout'] = df['timeout'] = df['time'].apply(lambda x: x>3600)
    df['is_solved'] = (~pd.isnull(df['error']))&(df['time']<=3600)
    return df

In [82]:
df_b_list = []
for b in benchmarks:
    df_b_list.append(process_benchmark(b))

df_b = pd.concat(df_b_list)
df_b.head()

Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
R

,index,tool,output,time,details,raw_output,lower,upper,error,error_norm,benchmark,id,is_timeout,timeout,is_solved
0,0,latte,"(nan, nan)",0.078246,[],NaN,NaN,NaN,NaN,NaN,rational_2,b0,False,False,False
1,1,latte,"(nan, nan)",0.058846,[],NaN,NaN,NaN,NaN,NaN,rational_2,b1,False,False,False
2,2,latte,"(nan, nan)",0.053086,[],NaN,NaN,NaN,NaN,NaN,rational_2,b2,False,False,False
3,3,latte,"(nan, nan)",0.048246,[],NaN,NaN,NaN,NaN,NaN,rational_2,b3,False,False,False
4,4,latte,"(nan, nan)",0.052024,[],NaN,NaN,NaN,NaN,NaN,rational_2,b4,False,False,False


In [83]:
def calcPlotID(bench):
    if bench.benchmark == 'rational':
        return 'A'+str(bench['index']+1)
    elif bench.benchmark == 'rational_2':
        return 'A'+str(bench['index']+30+1)
    if bench.benchmark == 'sqrt':
        return 'B'+str(bench['index']+1)
    elif bench.benchmark == 'rational_sqrt':
        return 'B'+str(bench['index']+30+1)
    
    return 0


df_b['plotID'] = df_b.apply(calcPlotID, axis=1)
df_b['catID'] = df_b['plotID'].str[:1]

df_b.head()

,index,tool,output,time,details,raw_output,lower,upper,error,error_norm,benchmark,id,is_timeout,timeout,is_solved,plotID,catID
0,0,latte,"(nan, nan)",0.078246,[],NaN,NaN,NaN,NaN,NaN,rational_2,b0,False,False,False,A31,A
1,1,latte,"(nan, nan)",0.058846,[],NaN,NaN,NaN,NaN,NaN,rational_2,b1,False,False,False,A32,A
2,2,latte,"(nan, nan)",0.053086,[],NaN,NaN,NaN,NaN,NaN,rational_2,b2,False,False,False,A33,A
3,3,latte,"(nan, nan)",0.048246,[],NaN,NaN,NaN,NaN,NaN,rational_2,b3,False,False,False,A34,A
4,4,latte,"(nan, nan)",0.052024,[],NaN,NaN,NaN,NaN,NaN,rational_2,b4,False,False,False,A35,A


In [84]:
df_b['tool'].unique()

array(['latte', 'alpha', 'gubpi', 'volesti', 'mathematica', 'wmilp',
       'psi'], dtype=object)

In [85]:
df_b.groupby(['tool']).agg({
    'time': ['mean', 'max'],
    'error': ['mean', 'max'],
    
    'is_solved': ['count', 'sum'],
    'timeout': ['sum'],
}).round(3)

time               error            is_solved      timeout
                mean       max      mean        max     count  sum     sum
tool                                                                      
alpha          3.922     5.983       0.0        0.0       120  120       0
gubpi          0.494     0.720  3.671111  31.569187       120  113       0
latte          0.020     0.109       NaN        NaN       120    0       0
mathematica   79.900  2326.064       0.0        0.0       120   61       0
psi          787.480  3626.377       0.0          0       120   20      20
volesti        0.048     0.132   0.51587   3.645832       120   60       0
wmilp        644.727  3623.372  0.112164   0.867839       120  107      13

In [86]:
# Define a function to classify the output type
def classify_output(row):
    if row.tool in ["psi", "mathematica"]:
        if pd.isnull(row.upper):
            if row.is_timeout:
                return "timeout"
            elif 'integrate' in str(row.raw_output).lower():
                return "symbolicoutput"
            elif 'complex' in str(row.raw_output).lower() and row.tool == "mathematica":
                return "complexoutput"
            else:
                return "toolerror"
        else:
            return "numeric"
    elif row.tool in ["volesti", "latte"]:
        if row.benchmark in ['rational_sqrt', 'sqrt']:
            if row.tool == "volesti":
                return "ns"
            elif row.tool == "latte":
                return "ns"
        elif row.benchmark in ['rational', 'rational_2']:
            if row.tool == "volesti":
                return "numeric"
            elif row.tool == "latte":
                return "toolerror"
    elif row.tool in ["gub", "gubpi"]:
        if pd.isnull(row.upper):
            return "toolerror"
        else:
            return "numeric"
    elif row.tool in ["wmilp", "alpha"]:
        return "numeric"
    return "unknown"

# Apply the classification function
df_b['output_type'] = df_b.apply(classify_output, axis=1)

# Group by tool and output type, and count the samples
stat_table = df_b.groupby(['tool', 'output_type']).size().reset_index(name='count')

# Display the table
stat_table

,tool,output_type,count
0,alpha,numeric,120
1,gubpi,numeric,113
2,gubpi,toolerror,7
3,latte,ns,60
4,latte,toolerror,60
5,mathematica,complexoutput,21
6,mathematica,numeric,61
7,mathematica,symbolicoutput,34
8,mathematica,toolerror,4
9,psi,numeric,20


On 120 benchmarks, PSI solved 20, timed out on 20, and failed to solve 80 (results contained integrals or were unable to evaluate the expression to a numeric value).
